# Enrich data with supermarket

In [2]:
# Libraries
import pandas as pd
import requests
import time
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pickle

In [3]:
import time
import pandas as pd
import requests
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Geolocator initialisieren
geolocator = Nominatim(user_agent="apartment_model")

def geocode_address(address, retries=3):
    """Return the (latitude, longitude) for a given address with retries."""
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address, timeout=10)
            if location:
                return location.latitude, location.longitude
        except Exception as e:
            print(f"Attempt {attempt+1} geocoding error for {address}: {e}")
            time.sleep(2)
    return None, None

def query_overpass(lat, lon, amenity="hospital", radius=1000, retries=3):
    """Query Overpass API for hospitals within radius with retries."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json][timeout:25];
    (
      node["amenity"="{amenity}"](around:{radius},{lat},{lon});
      way["amenity"="{amenity}"](around:{radius},{lat},{lon});
      relation["amenity"="{amenity}"](around:{radius},{lat},{lon});
    );
    out center;
    """
    for attempt in range(retries):
        try:
            response = requests.get(overpass_url, params={'data': query}, timeout=30)
            response.raise_for_status()
            return response.json().get("elements", [])
        except requests.RequestException as e:
            print(f"Attempt {attempt+1} Overpass API error ({amenity}): {e}")
            time.sleep(5)
    return []

def get_nearest_distance(lat, lon, amenity="hospital", radius=1000):
    """Return the minimum distance (in meters) from the given point to the nearest hospital."""
    elements = query_overpass(lat, lon, amenity, radius)
    distances = []
    for elem in elements:
        point = (elem.get('lat', elem.get('center', {}).get('lat')),
                 elem.get('lon', elem.get('center', {}).get('lon')))
        if None not in point:
            distances.append(geodesic((lat, lon), point).meters)
    return min(distances) if distances else None

# Load CSV file
df = pd.read_csv("original_apartment_data_analytics_hs24_with_lat_lon.csv", encoding='utf-8')

# Prepare new columns
df['latitude'] = pd.NA
df['longitude'] = pd.NA
df['distance_to_hospital'] = pd.NA

# Process each apartment
for idx, row in df.iterrows():
    full_address = f"{row['address']}, {row['town']}"
    lat, lon = geocode_address(full_address)
    if lat and lon:
        df.at[idx, 'latitude'] = lat
        df.at[idx, 'longitude'] = lon

        hospital_distance = get_nearest_distance(lat, lon, "hospital")
        df.at[idx, 'distance_to_hospital'] = hospital_distance if hospital_distance is not None else "Not found"

        print(hospital_distance)
    else:
        print(f"Geocoding failed for address: {full_address}")

    # Extended sleep to avoid API rate limits
    time.sleep(1.5)

# Save result
df.to_csv("extended_apartment_data_hospital.csv", index=False, encoding="utf-8")
print("CSV file successfully extended and saved as 'extended_apartment_data_hospital.csv'")


None
None
None
None
None
None
None
None
None
None
None
821.2456099148582
821.2456099148582
None
None
821.2456099148582
None
None
None
None
None
None
None
None
None
821.2456099148582
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
821.2456099148582
None
None
None
None
None
609.2604137907805
None
None
None
1109.4126407408974
None
1109.4126407408974


KeyboardInterrupt: 

In [ ]:
#Load the extended CSV file
df = pd.read_csv("extended_apartment_data_hospital.csv", encoding='utf-8')

#Remove all rows where 'distance_to_hospital' is "Not found"
df = df[df['distance_to_hospital'] != "Not found"]

#Save the cleaned data back to a CSV file
df.to_csv("cleaned_apartment_data_hospital.csv", index=False, encoding='utf-8')

print("Die CSV-Datei wurde erfolgreich als Pickle-Datei gespeichert!")


Die CSV-Datei wurde erfolgreich als Pickle-Datei gespeichert!
